In [ ]:
import easyocr
import cv2
import pytesseract
import re
import time
import numpy as np
import matplotlib.pyplot as plt 
from datetime import datetime

In [ ]:
# import sql function from sql_connection.py filwe using magic tool(%run)
%run sql_connection.py


In [ ]:
# setting path for pytesseract model from the file directory
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
# initializing object of easyocr, set english as target language and set gpu as False 
reader = easyocr.Reader(['en'],gpu=False)

In [ ]:
# creating object for number plate detection model 
numplate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

In [ ]:
# function to extract number plate 
def extract_plate(frame):
    # conveting BGR frame to gray
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    # Sharpen the image using the Laplacian operator 
    sharpened_image2 = cv2.GaussianBlur(gray, (7, 7), 0)
    # detect number plate using pre-defined model 
    num_plate = numplate_cascade.detectMultiScale(sharpened_image2,minNeighbors=10)
    if len(num_plate) > 0:
        W,H,w,h = num_plate[0]
        cv2.rectangle(frame,(W,H),(W+w,H+h),(0,0,200),2)

        plate = frame[H:H+h,W:W+w]
        plt.imshow(plate)
        plt.show()
        return plate
    else:
        None

In [ ]:
# function to remove black spaces and capitalizing the letters
def upper_strip(num_plate):
    return ''.join(num_plate.split()).upper()

In [ ]:
# function to extract character from the number plate 
def extract_number(plate):
    # extracting character by implementing pytesseract
    # reg_num = pytesseract.image_to_string(plate,output_type= pytesseract.Output.DICT)
    # extacting characters from the extracted frame using easyocr and detail =0 will retrun text only
    reg_num = reader.readtext(plate,detail=0)
    # pattern to retrun number plate, ex: KL09H9999
    pattern = r'^[A-Z]{2}.*[0-9]{1,4}$'
    matches = []
    for i in reg_num:
        # extracted number plate will be compared with the pattern and matching pattern will be inserted to 'matches' list
        match = re.findall(pattern, upper_strip(i))
        if match:
            matches.extend(match)
    
    if matches:
        # each frame will only contain one number plate  
        return matches[0]
    else:
        return None
    



In [ ]:
create_table() # creating table using query from sql_coonection.py

In [ ]:
# dictionary to store vehicle entering the parking space and time of entry
plate_entry_times = {}

In [ ]:
# Open a video file for processing
cap = cv2.VideoCapture(r'd:\Deep Learning\OCR(Object_character_recognition)\WhatsApp Video 2024-06-28 at 11.48.16_00aa7520.mp4')

# Initialize frame count
count = 0

# Initialize a dictionary to keep track of plate entry times
plate_entry_times = {}

# Loop through each frame of the video
while True:
    # Read a frame from the video
    ret, frame = cap.read()

    # If the frame was not read correctly, exit the loop
    if not ret:
        break

    # Resize the frame to half its original size
    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)

    # Extract the number plate from the frame
    number_plate = extract_plate(frame)

    # If a number plate is detected
    if number_plate is not None:
        # Extract the registration number from the number plate
        reg_num = extract_number(number_plate)

        # Overlay the registration number on the frame
        cv2.putText(frame, f'{reg_num}', (100, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (0, 0, 200), 2)

        # Perform actions every 10 frames
        if count % 10 == 0:
            print(f"Frame count {count}")

            # If a registration number is extracted
            if reg_num:
                # If the registration number is not already logged in plate_entry_times
                if reg_num not in plate_entry_times.keys():
                    # Record the entry timestamp
                    timestamp_entering = datetime.now()

                    # Insert entry details into the database and get the entry ID
                    entry_id = insert_entry(reg_num, timestamp_entering)

                    # Store the entry timestamp in the dictionary
                    plate_entry_times[reg_num] = datetime.now()

                # If an entry ID exists (i.e., the car was logged in)
                if entry_id is not None:
                    # Record the exit timestamp
                    timestamp_leaving = datetime.now()

                    # Calculate the time taken for parking
                    time_taken = timestamp_leaving - timestamp_entering

                    # Calculate the parking fee based on entry and exit times
                    parking_fee = calculate_parking_fee(timestamp_entering, timestamp_leaving)

                    # Update the database with exit details
                    update_entry(entry_id, timestamp_leaving, time_taken, parking_fee)

                    # Remove the registration number from the dictionary
                    del plate_entry_times[reg_num]

    # Increment the frame count
    count += 1

    # Display the frame with the overlayed registration number
    cv2.imshow('frame', frame)

    # Exit the loop if the 'k' key is pressed
    if cv2.waitKey(100) == ord('k'):
        break

# Release the video capture object and close all OpenCV windows
cv2.destroyAllWindows()
cap.release()

In [ ]:

cap =cv2.VideoCapture(r'd:\Deep Learning\OCR(Object_character_recognition)\WhatsApp Video 2024-06-28 at 11.48.16_00aa7520.mp4')
count=0
while True:
    ret,frame = cap.read()

    if not ret:
        break 
    frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)
    number_plate = extract_plate(frame)
    

    if  number_plate is not None:
        reg_num = extract_number(number_plate)
        
        cv2.putText(frame,f'{reg_num}',(100,50),cv2.FONT_HERSHEY_COMPLEX_SMALL,2,(0,0,200),2)
        if count%10 == 0:
            print(f"Frame count {count}")
            if reg_num:
                if reg_num not in plate_entry_times.keys():
                    timestamp_entering = datetime.now()
                    entry_id = insert_entry(reg_num, timestamp_entering)
                    plate_entry_times[reg_num] =datetime.now()
                    
                if entry_id is not None:
                     timestamp_leaving = datetime.now()
                     time_taken = timestamp_leaving - timestamp_entering
                     parking_fee = calculate_parking_fee(timestamp_entering, timestamp_leaving)
                     update_entry(entry_id, timestamp_leaving, time_taken, parking_fee)
                    
                     del plate_entry_times[reg_num]
                    
    count+=1
    cv2.imshow('frame',frame)
    if cv2.waitKey(100) == ord('k'):
            break
cv2.destroyAllWindows()
cap.release()